In [300]:
import sys
import os
import re
import pandas as pd

current_directory = os.getcwd()
logfolder_path = os.path.join(current_directory, 'logfiles')
log_files = os.listdir(logfolder_path)
log_files = [file for file in log_files if file.endswith('.txt')]

log_files[4148], len(log_files)

("HH20250210 Aludra #6 - $0.05-$0.10 - USD No Limit Hold'em.txt", 4344)

In [301]:
tablegame = log_files.index("HH20250210 Aludra #6 - $0.05-$0.10 - USD No Limit Hold'em.txt")
tablegame

4148

In [302]:
#Variables

#count variables
game = 1
row_number = 0
hand = 0
won=0

#state variables
start_hand = 0
preflop = 0
flop = 0
turn = 0
river = 0
showdown = 0

#player variables
dealer_position = 0
seats = {}
cards = []
player_name_post_blind = ""
blind = 0
action = []
flopcards = []
turncard = []
rivercard = []

#action variables
preflop_fold_dict = {}
preflop_bet_dict = {}
preflop_check_dict = {}
flop_fold_dict = {}
flop_bet_dict = {}
flop_check_dict = {}
turn_fold_dict = {}
turn_bet_dict = {}
turn_check_dict = {}
river_fold_dict = {}
river_bet_dict = {}
river_check_dict = {}

pd_dataset = pd.DataFrame()


In [303]:
#re patterns
pattern_start = r"PokerStars"
pattern_preflop = r"HOLE CARDS"
pattern_flop = r"FLOP"
pattern_turn = r"TURN"
pattern_river = r"RIVER"
pattern_showdown = r"SHOW DOWN"
pattern_dealer_position = r"Seat #(\d+)"
pattern_player_information = r"Seat (\d+): ([\w\-\u0080-\uFFFF\s]+) \((\d+) in chips\)"
pattern_player_information_cashgame = r"Seat (\d+): ([\w\-\u0080-\uFFFF\s]+) \(\$(\d+(?:\.\d{1,2})?) in chips\)"
pattern_post_blinds = r"([\w\-\u0080-\uFFFF\s]+): posts .* \$(\d+\.\d{2})"
pattern_handcards = r"Dealt to \S+ \[([A2-9TJQKA][cdhs])( [A2-9TJQKA][cdhs])*\]"
pattern_flop_cards = r"\[([A2-9TJQKA][cdhs]) ([A2-9TJQKA][cdhs]) ([A2-9TJQKA][cdhs])\]"
pattern_turn_card = r"\[([A2-9TJQKA][cdhs])\]"
pattern_river_card = r"\[([A2-9TJQKA][cdhs])\]"
pattern_summary = r"SUMMARY"

In [304]:
# Function to handle extraction of bets, folds, raises, checks, and calls
def extract_action(line):
    # Regex patterns for each action
    fold_pattern = r"([\w\-\u0080-\uFFFF\s]+): folds"
    bet_pattern = r"([\w\-\u0080-\uFFFF\s]+): bets \$(\d+\.\d{1,2})"
    raise_pattern = r"([\w\-\u0080-\uFFFF\s]+): raises \$(\d+\.\d{1,2}) to \$(\d+\.\d{1,2})"
    check_pattern = r"([\w\-\u0080-\uFFFF\s]+): checks"
    call_pattern = r"([\w\-\u0080-\uFFFF\s]+): calls \$(\d+\.\d{1,2})"  # New pattern for calls
    collection_pattern = r"([\w\-\u0080-\uFFFF\s]+) collected \$(\d+\.\d{1,2}) from pot"

    # Check for fold action
    fold_match = re.search(fold_pattern, line,  re.UNICODE)
    if fold_match:
        player_name = fold_match.group(1)
        return [player_name, 'fold' , 0]

    # Check for bet action
    bet_match = re.search(bet_pattern, line,  re.UNICODE)
    if bet_match:
        player_name = bet_match.group(1)
        bet = float(bet_match.group(2))
        return [player_name, 'bet', bet]

    # Check for raise action
    raise_match = re.search(raise_pattern, line,  re.UNICODE)
    if raise_match:
        player_name = raise_match.group(1)
        bet = float(raise_match.group(3))  # This is the second amount (raise to)
        return [player_name, 'raise', bet]

    # Check for check action
    check_match = re.search(check_pattern, line,  re.UNICODE)
    if check_match:
        player_name = check_match.group(1)
        return [player_name, 'check', 0]

    # Check for call action
    call_match = re.search(call_pattern, line,  re.UNICODE)
    if call_match:
        player_name = call_match.group(1)
        bet = call_match.group(2)
        return [player_name, 'bet', bet]
    
    # Check for collection action
    collection_match = re.search(collection_pattern, line,  re.UNICODE)
    if collection_match:
        player_name = collection_match.group(1)
        collected_amount = float(collection_match.group(2))
        return [player_name, 'collect', collected_amount]
    
    return "No action found"   


In [305]:
# function to set values to player variables

In [306]:
first_txt_file = log_files[tablegame]
first_txt_file_path = os.path.join(logfolder_path, first_txt_file)

with open(first_txt_file_path, 'r') as file:
    for line in file:
        row_number += 1 
        has_content = bool(line.strip())  

        contains_start = bool(re.search(pattern_start, line))
        contains_holed_cards = bool(re.search(pattern_preflop, line))
        contains_flop = bool(re.search(pattern_flop, line))
        contains_turn = bool(re.search(pattern_turn, line))
        contains_river = bool(re.search(pattern_river, line))
        contains_showdown = bool(re.search(pattern_showdown, line))
        contains_summary = bool(re.search(pattern_summary, line))

        match = re.search(pattern_dealer_position, line,  re.UNICODE)
       
        #Adjust pattern for Spin and Go here
        match_player = re.search(pattern_player_information_cashgame, line, re.UNICODE)
        match_post_blinds = re.search(pattern_post_blinds, line,  re.UNICODE)
        match_handcards = re.search(pattern_handcards, line, re.UNICODE)

        
        #start of a hand; first set state variables
        if contains_start == True:
            start_hand = 1
            preflop = 0
            flop = 0
            turn = 0
            river = 0
            showdown = 0
        
        if contains_holed_cards == True:
            start_hand = 0
            preflop = 1
            flop = 0
            turn = 0
            river = 0
            showdown = 0

        if contains_flop == True:
            start_hand = 0
            preflop = 0
            flop = 1
            turn = 0
            river = 0
            showdown = 0
            contains_flop_cards = re.search(pattern_flop_cards, line, re.UNICODE)
            flopcards = [contains_flop_cards.group(1), contains_flop_cards.group(2), contains_flop_cards.group(3)]

        if contains_turn == True:
            start_hand = 0
            preflop = 0
            flop = 0
            turn = 1
            river = 0
            showdown = 0
            contains_turn_card = re.search(pattern_turn_card, line, re.UNICODE)
            turncard = contains_turn_card.group(1)

        if contains_river == True:
            start_hand = 0
            preflop = 0
            flop = 0
            turn = 0
            river = 1
            showdown = 0
            contains_river_card = re.search(pattern_river_card, line, re.UNICODE)
            rivercard = contains_river_card.group(1)

        if contains_showdown == True:
            start_hand = 0
            preflop = 0
            flop = 0
            turn = 0
            river = 0
            showdown = 1

        if has_content == False:
            row_number = 0
            start_hand = 0
            preflop = 0
            flop = 0
            turn = 0
            river = 0
            showdown = 0
            seats = {}
            cards = []
            flopcards = []
            turncard = []
            rivercard = []
            preflop_fold_dict = {}
            preflop_bet_dict = {}
            preflop_check_dict = {}
            flop_fold_dict = {}
            flop_bet_dict = {}
            flop_check_dict = {}
            turn_fold_dict = {}
            turn_bet_dict = {}
            turn_check_dict = {}
            river_fold_dict = {}
            river_bet_dict = {}
            river_check_dict = {}
            won = 0

        if row_number == 1:
            hand += 1

        # Extract player information
        if start_hand == 1 and match:
            dealer_position = match.group(1)

        if start_hand == 1 and match_player:
            seat_number = match_player.group(1)
            player_name = match_player.group(2)
            chips = match_player.group(3)

            # Assign the extracted details to the seat
            seats[player_name] = {'seat': seat_number, 'chips': chips}  

        if start_hand == 1 and match_post_blinds:
            player_name_post_blind = match_post_blinds.group(1)
            blind = float(match_post_blinds.group(2))
            seats[player_name_post_blind]['chips'] =  str(round(float(seats[player_name_post_blind]['chips']) - blind, 2))

        else:
            player_name_post_blind = ""
            blind = 0
      
        #Extract Preflop infromation

        if preflop == 1 and match_handcards:
            cards = match_handcards.group(0)[match_handcards.group(0).find('[')+1:match_handcards.group(0).find(']')].split()

        if preflop == 1:
            action = extract_action(line)
            if action[0] != 'N':
                if action[1] == 'fold':
                    preflop_fold_dict[action[0]] = {"position": seats[action[0]]['seat'], "fold": 1}
                elif action[1] == 'bet' or action[1] == 'raise':
                    if action[0] in preflop_bet_dict:
                        preflop_bet_dict[action[0]]["bet"] += float(action[2])
                    else:  
                        preflop_bet_dict[action[0]] = {"position": seats[action[0]]['seat'], "bet": float(action[2])}
                elif action[1] == 'check':
                    preflop_check_dict[action[0]] = {"position": seats[action[0]]['seat'], "check": 1}
                elif action[1] == 'collect' and action[0] == 'Mahu2021':
                    won = 1
            
        elif flop == 1:
            action = extract_action(line)
            if action[0] != 'N':
                if action[1] == 'fold':
                    flop_fold_dict[action[0]] = {"position": seats[action[0]]['seat'], "fold": 1}
                elif action[1] == 'bet' or action[1] == 'raise':
                    if action[0] in flop_bet_dict:
                        flop_bet_dict[action[0]]["bet"] += float(action[2])
                    else:  
                        flop_bet_dict[action[0]] = {"position": seats[action[0]]['seat'], "bet": float(action[2])}
                elif action[1] == 'check':
                    flop_check_dict[action[0]] = {"position": seats[action[0]]['seat'], "check": 1}
                elif action[1] == 'collect' and action[0] == 'Mahu2021':
                    won = 1

        elif turn == 1:
            action = extract_action(line)
            if action[0] != 'N':
                if action[1] == 'fold':
                    turn_fold_dict[action[0]] = {"position": seats[action[0]]['seat'], "fold": 1}
                elif action[1] == 'bet' or action[1] == 'raise':
                    if action[0] in turn_bet_dict:
                        turn_bet_dict[action[0]]["bet"] += float(action[2])
                    else:  
                        turn_bet_dict[action[0]] = {"position": seats[action[0]]['seat'], "bet": float(action[2])}
                elif action[1] == 'check':
                    turn_check_dict[action[0]] = {"position": seats[action[0]]['seat'], "check": 1}
                elif action[1] == 'collect' and action[0] == 'Mahu2021':
                    won = 1

        elif river == 1:
            action = extract_action(line)
            if action[0] != 'N':
                if action[1] == 'fold':
                    river_fold_dict[action[0]] = {"position": seats[action[0]]['seat'], "fold": 1}
                elif action[1] == 'bet' or action[1] == 'raise':
                    if action[0] in river_bet_dict:
                        river_bet_dict[action[0]]["bet"] += float(action[2])
                    else:  
                        river_bet_dict[action[0]] = {"position": seats[action[0]]['seat'], "bet": float(action[2])}
                elif action[1] == 'check':
                    river_check_dict[action[0]] = {"position": seats[action[0]]['seat'], "check": 1}
                elif action[1] == 'collect' and action[0] == 'Mahu2021':
                    won = 1
        
        elif showdown == 1:
            action = extract_action(line)
            if action[0] != 'N':
                if action[1] == 'collect' and action[0] == 'Mahu2021':
                    won = 1

        else:
            action = []
        
        if contains_summary == True:
            print(won ,row_number, hand, cards, flopcards, turncard, rivercard, seats, preflop, flop, turn, river, showdown, preflop_fold_dict, preflop_bet_dict, preflop_check_dict, flop_fold_dict, flop_bet_dict, flop_check_dict, turn_fold_dict, turn_bet_dict, turn_check_dict, river_fold_dict, river_bet_dict, river_check_dict)

1 21 1 ['8s', '2s'] [] [] [] {'TobiasA1982': {'seat': '1', 'chips': '10'}, '00O-2Z': {'seat': '2', 'chips': '10.0'}, 'Mahu2021': {'seat': '3', 'chips': '9.9'}, 'acebob999': {'seat': '4', 'chips': '13.85'}, 'Chakman1990': {'seat': '5', 'chips': '10'}, 'lolb0t': {'seat': '6', 'chips': '16.12'}} 1 0 0 0 0 {'acebob999': {'position': '4', 'fold': 1}, 'Chakman1990': {'position': '5', 'fold': 1}, 'lolb0t': {'position': '6', 'fold': 1}, 'TobiasA1982': {'position': '1', 'fold': 1}, '00O-2Z': {'position': '2', 'fold': 1}} {} {} {} {} {} {} {} {} {} {} {}
1 28 2 ['3h', '5h'] ['2h', '5c', '8h'] [] [] {'JilKaGen': {'seat': '1', 'chips': '30.25'}, 'Mahu2021': {'seat': '2', 'chips': '10.0'}, 'lolb0t': {'seat': '3', 'chips': '26.1'}, 'TANEV78': {'seat': '4', 'chips': '19.74'}, 'Tozsy17': {'seat': '5', 'chips': '12.08'}, 'acebob999': {'seat': '6', 'chips': '13.85'}} 0 1 0 0 0 {'TANEV78': {'position': '4', 'fold': 1}, 'Tozsy17': {'position': '5', 'fold': 1}, 'acebob999': {'position': '6', 'fold': 1}, 'J